# Permutation Flow-Shop Scheduling Problem

This is a variant of the Flot-shop scheduling problem (FSSP) in which the sequence of jobs is the same in every machine.

$$
 \begin{align}
     \text{min} \quad & C_{\text{max}} \\
     \text{s.t.} \quad & x_{m-1, j} + p_{m, j} \leq x_{m, j}
         & \forall ~ j \in J; m \in (2, ..., |M|)\\
     & x_{m, j} + p_{m, j} \leq x_{m, k} \lor x_{m, k} + p_{m, k} \leq x_{m, j}
         & \forall ~ j \in J; k \in J, j \neq k\\
     & x_{|M|, j} + p_{|M|, j} \leq C_{\text{max}}
         & \forall ~ j \in J\\
     & x_{m, j} \geq 0 & \forall ~ j \in J; m \in M\\
     & z_{j, k} \in \{0, 1\} & \forall ~ j \in J; k \in J, j \neq k\\
 \end{align}
 $$

 You can compare this implementation to MILP solvers at the [end of the notebook](#bonus---milp-model).

In [1]:
import json
import time

import pandas as pd

from bnbprob.pfssp import CallbackBnB, PermFlowShop
from bnbprob.pfssp.cython.problem import PermFlowShopQuit
from bnbpy import configure_logfile

In [2]:
configure_logfile("pfssp-experiments.log", mode="w")

In [3]:
solutions = {}
# problems = [f'ta{str(i).zfill(3)}' for i in range(41, 51, 1)]
problems = (
    [f'ta{str(i).zfill(3)}' for i in range(11, 21, 1)]
    + [f'ta{str(i).zfill(3)}' for i in range(41, 51, 1)]
)

bnbs = {}
solutions = {}

for prob in problems:
    with open(
        f'./../data/flow-shop/{prob}.json', mode='r', encoding='utf8'
    ) as f:
        p = json.load(f)
        problem = PermFlowShopQuit.from_p(p, constructive='neh')
        bnb = CallbackBnB(
            eval_node='in',
            rtol=0.0001,
            heur_factor=100,
            save_tree=False,
            queue_mode='dfs',  # 'dfs' or 'cycle'
        )
        bnb.log_row(prob)

        start = time.time()
        sol = bnb.solve(problem, maxiter=100_000_000, timelimit=3600)
        duration = time.time() - start
        solutions[prob] = (sol, duration)

        print(f"{prob}: {sol} in {duration:.2f}s")

        jobs = [j.j for j in sol.sequence]
        solutions[prob] = jobs

        bnbs[prob] = bnb
        time.sleep(0.1)

ta011: Status: OPTIMAL | Cost: 1582.0 | LB: 1582.0 in 9.03s
ta012: Status: OPTIMAL | Cost: 1659.0 | LB: 1659.0 in 34.02s
ta013: Status: OPTIMAL | Cost: 1496.0 | LB: 1496.0 in 15.73s
ta014: Status: OPTIMAL | Cost: 1377.0 | LB: 1377.0 in 3.06s
ta015: Status: OPTIMAL | Cost: 1419.0 | LB: 1419.0 in 6.64s
ta016: Status: OPTIMAL | Cost: 1397.0 | LB: 1397.0 in 8.81s
ta017: Status: OPTIMAL | Cost: 1484.0 | LB: 1484.0 in 825.71s
ta018: Status: OPTIMAL | Cost: 1538.0 | LB: 1538.0 in 23.22s
ta019: Status: OPTIMAL | Cost: 1593.0 | LB: 1593.0 in 2.22s
ta020: Status: OPTIMAL | Cost: 1591.0 | LB: 1591.0 in 30.74s
ta041: Status: OPTIMAL | Cost: 2991.0 | LB: 2991.0 in 247.20s
ta042: Status: FEASIBLE | Cost: 2893.0 | LB: 2829.0 in 3600.01s
ta043: Status: FEASIBLE | Cost: 2865.0 | LB: 2824.0 in 3600.00s
ta044: Status: OPTIMAL | Cost: 3063.0 | LB: 3063.0 in 2885.79s
ta045: Status: OPTIMAL | Cost: 2976.0 | LB: 2976.0 in 333.50s
ta046: Status: OPTIMAL | Cost: 3006.0 | LB: 3006.0 in 144.10s
ta047: Status: FE

In [4]:
with open('pfssp-solutions.json', mode='w', encoding='utf8') as f:
    json.dump(
        solutions,
        f,
        indent=4,
    )

In [5]:
# Export to dataframe
problems_ = []
ub = []
lb = []
gap = []
time_ = []
n_nodes = []

for key, val in solutions.items():
    sol, t = val
    problems_.append(key)
    ub.append(sol.cost)
    lb.append(sol.lb)
    gap.append(abs(sol.cost - sol.lb) / sol.cost)
    time_.append(t)


df = pd.DataFrame({
    "problem": problems_,
    "ub": ub,
    "lb": lb,
    "gap": gap,
    "time": time_
})

df.to_excel("bench-pfssp-quit-dfs.xlsx")

ValueError: too many values to unpack (expected 2)